In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
import pygame

import torch
import torch.nn as nn
import torch.nn.functional as F

from math import *
from IPython import display

import time

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
im = np.array(Image.open("/home/nichg/Dropbox/Pictures/IMG_0822.JPG"))
im = im[::8,::8]

print(im.shape)

(324, 432, 3)


In [2]:
XRES = 256
YRES = 256
MULT = 1

pygame.init()
screen = pygame.display.set_mode((XRES, YRES), 0, 24)

In [30]:
def getFitness(target, current, updates, pos):
    cost = 0
    benefit = 0
    curpos = pos.copy()
    im = current.copy()
    for i in range(len(updates)):
        curpos[0] = (curpos[0] + XRES + updates[i][0])%XRES
        curpos[1] = (curpos[1] + YRES + updates[i][1])%YRES
        x = int(curpos[0])
        y = int(curpos[1])
        
        if x>=0 and y>=0 and x<XRES and y<YRES:
            benefit += np.sum(np.abs(target[x,y]-im[x,y]))
            im[x,y] = target[x,y]
    
    return benefit# - cost

def getUpdates(target, current, pos, N=100, steps=100):
    updates = np.random.randint(5, size=(N,2))-2
    fitness = getFitness(target, current, updates, pos)
    
    for step in range(steps):
        newupdates = updates.copy()
        newupdates[np.random.randint(newupdates.shape[0])] = np.random.randint(5, size=(2,))-2
        
        newfitness = getFitness(target, current, newupdates, pos)
        
        if newfitness>fitness:
            fitness = newfitness
            updates = newupdates
        
    return updates

In [ ]:
#!mkdir output3
#!rm output3/*.png

t0 = 0
frame = 0
view = np.zeros((XRES,YRES,3))
ant = np.array([XRES/2, YRES/2])
glitch = 0

with torch.no_grad():
    while True:
        for event in pygame.event.get():
            pass
        
        im = np.array(Image.open("/home/nichg/Dropbox/art/logo/%.4d.png" % (1+frame%100)))[:,:,:3]
        
        for i in range(100):
            updates = getUpdates(im, view, ant, N=10, steps=10)
            for u in updates:
                ant[0] = (ant[0] + u[0] + XRES)%XRES
                ant[1] = (ant[1] + u[1] + YRES)%YRES
                x = int(ant[0])
                y = int(ant[1])
                xm = np.clip(x-1,0,XRES-1)
                ym = np.clip(y-1,0,YRES-1)
                
                view[xm:x+2, ym:y+2] = im[xm:x+2, ym:y+2]
                
        view *= 0.995
        #view = np.clip((0.999*view*(1-r[:,:,3:]) + 1.4*r[:,:,:3]*r[:,:,3:]),0,255)
        #im = Image.fromarray(view.astype(np.uint8))
        #im.save("output3/%.6d.png" % frame)
        s_image = pygame.pixelcopy.make_surface(view.astype(np.uint8).transpose(1,0,2))

        screen.blit(s_image, (0, 0))

        pygame.display.update()   
        time.sleep(0.005)
        
        frame += 1